In [31]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

In [32]:
from api_keys import g_key

In [33]:
output_data_file = "Weather_Cities.csv"
cities_wth_2 = pd.read_csv(output_data_file)
cities_wth_2

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Rikitea,100,PF,1570048732,86,-23.12,-134.97,61.86,10.13
1,Kekri,0,IN,1570048981,82,25.97,75.15,75.30,6.42
2,Saint-Philippe,90,CA,1570048745,62,45.36,-73.48,55.00,12.75
3,Bluff,0,AU,1570048766,76,-23.58,149.07,62.32,14.56
4,Hobart,0,AU,1570048546,66,-42.88,147.33,53.01,9.17
...,...,...,...,...,...,...,...,...,...
550,Aswan,0,EG,1570048876,21,24.09,32.90,82.34,14.52
551,Abu Kamal,0,SY,1570049202,30,34.45,40.92,79.51,13.82
552,Palu,77,ID,1570049202,91,-0.90,119.87,72.51,3.78
553,Haapiti,20,PF,1570049202,65,-17.57,-149.87,80.60,4.70


In [34]:
gmaps.configure(api_key=g_key)

In [35]:
locations = cities_wth_2[["Lat", "Lng"]]
humidity = cities_wth_2["Humidity"].astype(float)

In [36]:
fig = gmaps.figure(center=(20,0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=600)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
ideal_wth = cities_wth_2.loc[(cities_wth_2["Max Temp"] < 80) & (cities_wth_2["Max Temp"] > 70) &
                             (cities_wth_2["Wind Speed"] < 10) & (cities_wth_2["Cloudiness"] == 0)]
ideal_wth = pd.DataFrame(ideal_wth)
ideal_wth.dropna()
ideal_wth

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
1,Kekri,0,IN,1570048981,82,25.97,75.15,75.30,6.42
28,Kupang,0,ID,1570048994,70,-10.16,123.60,75.95,2.59
121,Nanchang,0,CN,1570049020,73,28.68,115.89,70.53,1.03
143,Parola,0,IN,1570049029,90,20.88,75.11,75.64,5.75
184,Mount Isa,0,AU,1570048776,33,-20.73,139.49,71.60,4.70
241,Ponta do Sol,0,BR,1570048778,44,-20.63,-46.00,78.56,4.27
259,Maragogi,0,BR,1570048763,85,-9.01,-35.22,72.26,5.70
274,Basna,0,IN,1570049083,78,25.97,73.75,79.23,9.89
292,Tikapur,0,NP,1570049089,91,28.53,81.12,72.06,2.26
296,Qinzhou,0,CN,1570049090,80,21.96,108.62,73.79,2.35


In [38]:
ideal_wth.count()


City          19
Cloudiness    19
Country       19
Date          19
Humidity      19
Lat           19
Lng           19
Max Temp      19
Wind Speed    19
dtype: int64

In [39]:
ideal_wth['Hotel Name'] = ""
ideal_wth

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
1,Kekri,0,IN,1570048981,82,25.97,75.15,75.30,6.42,
28,Kupang,0,ID,1570048994,70,-10.16,123.60,75.95,2.59,
121,Nanchang,0,CN,1570049020,73,28.68,115.89,70.53,1.03,
143,Parola,0,IN,1570049029,90,20.88,75.11,75.64,5.75,
184,Mount Isa,0,AU,1570048776,33,-20.73,139.49,71.60,4.70,
241,Ponta do Sol,0,BR,1570048778,44,-20.63,-46.00,78.56,4.27,
259,Maragogi,0,BR,1570048763,85,-9.01,-35.22,72.26,5.70,
274,Basna,0,IN,1570049083,78,25.97,73.75,79.23,9.89,
292,Tikapur,0,NP,1570049089,91,28.53,81.12,72.06,2.26,
296,Qinzhou,0,CN,1570049090,80,21.96,108.62,73.79,2.35,


In [40]:
params = {
    "radius": 50000,
    "types": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in ideal_wth.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_hotel = requests.get(base_url, params=params)
    

    # convert to json
    name_hotel = name_hotel.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        ideal_wth.loc[index, "Hotel Name"] = name_hotel["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [41]:
hotel_df = pd.DataFrame(ideal_wth)
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
1,Kekri,0,IN,1570048981,82,25.97,75.15,75.30,6.42,Kekri
28,Kupang,0,ID,1570048994,70,-10.16,123.60,75.95,2.59,Kupang
121,Nanchang,0,CN,1570049020,73,28.68,115.89,70.53,1.03,Nanchang
143,Parola,0,IN,1570049029,90,20.88,75.11,75.64,5.75,Dhule
184,Mount Isa,0,AU,1570048776,33,-20.73,139.49,71.60,4.70,Cloncurry
241,Ponta do Sol,0,BR,1570048778,44,-20.63,-46.00,78.56,4.27,Piumhi
259,Maragogi,0,BR,1570048763,85,-9.01,-35.22,72.26,5.70,Ipojuca
274,Basna,0,IN,1570049083,78,25.97,73.75,79.23,9.89,Pali
292,Tikapur,0,NP,1570049089,91,28.53,81.12,72.06,2.26,Gulariya
296,Qinzhou,0,CN,1570049090,80,21.96,108.62,73.79,2.35,Nanning


In [42]:
gmaps.configure(api_key=g_key)

locations = hotel_df[["Lat", "Lng"]]
humidity = hotel_df["Humidity"].astype(float)

fig = gmaps.figure(center=(20,0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=50)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [43]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row)for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

hotel_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)


fig = gmaps.figure(center=(20,0), zoom_level=2)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [44]:
fig = gmaps.figure(center=(20,0), zoom_level=2)

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [44]:
fig = gmaps.figure(center=(20,0), zoom_level=2)

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))